In [1]:
import numpy as np
import pandas as pd
from edward.models import Gamma
import edward as ed
import tensorflow as tf
import matplotlib.pyplot as plt
#from weibull import *
%run weibull.py
%matplotlib inline

In [2]:
P = 1
N = 100
w2 = TruncatedWeibull(shape = 1 * tf.ones(P), scale = 1 * tf.ones(P), cens = 2 * tf.ones(P))

In [3]:
sess = tf.Session()

with sess.as_default():
    y = w2.sample(100).eval().reshape(-1)

In [4]:
y[0:5]

array([ 3.25422907,  2.31532216,  3.17995644,  3.56822944,  2.15718436], dtype=float32)

In [5]:
## Survival times
k = Gamma(alpha = tf.ones(1), beta = tf.ones(1)) # Weibull hyperparameter
p = Gamma(alpha = tf.ones(1), beta = tf.ones(1)) # Weibull hyperparameter

censoring_times = 4 * tf.ones(P)

In [6]:
x = TruncatedWeibull(shape = k * tf.ones(N), scale = p * tf.ones(N), cens = censoring_times)

x2 = TruncatedWeibull2(shape = k * tf.ones(N), scale = p * tf.ones(N))
x2._cens = censoring_times

In [7]:
ds = tf.contrib.distributions
qk = ed.models.TransformedDistribution(
    distribution = ed.models.NormalWithSoftplusSigma(mu = tf.Variable(tf.zeros(1)),
                                                    sigma = tf.Variable(tf.ones(1))),
    bijector = ds.bijector.Exp(),
    name = "qk"
)

qp = ed.models.TransformedDistribution(
    distribution = ed.models.NormalWithSoftplusSigma(mu = tf.Variable(tf.zeros(1)),
                                                    sigma = tf.Variable(tf.ones(1))),
    bijector = ds.bijector.Exp(),
    name = "qp"
)

In [8]:
# %run weibull.py
approx_dict = {
    k: qk,
    p: qp
}

data_dict = {
    x: y
}



In [9]:
losses_1 = []

for _ in range(100):
    inference = ed.KLqp(approx_dict, data = data_dict)
    inference.run()
    losses_1.append(inference.loss.eval())

1000/1000 [100%] ██████████████████████████████ Elapsed: 4s | Loss: nan
1000/1000 [100%] ██████████████████████████████ Elapsed: 5s | Loss: nan
1000/1000 [100%] ██████████████████████████████ Elapsed: 4s | Loss: nan
1000/1000 [100%] ██████████████████████████████ Elapsed: 5s | Loss: -49.581
1000/1000 [100%] ██████████████████████████████ Elapsed: 4s | Loss: 145.255
1000/1000 [100%] ██████████████████████████████ Elapsed: 4s | Loss: nan
1000/1000 [100%] ██████████████████████████████ Elapsed: 4s | Loss: -69.757
1000/1000 [100%] ██████████████████████████████ Elapsed: 4s | Loss: nan
1000/1000 [100%] ██████████████████████████████ Elapsed: 5s | Loss: 413.931
1000/1000 [100%] ██████████████████████████████ Elapsed: 4s | Loss: 361.592
1000/1000 [100%] ██████████████████████████████ Elapsed: 4s | Loss: 530.339
1000/1000 [100%] ██████████████████████████████ Elapsed: 5s | Loss: 85.806
1000/1000 [100%] ██████████████████████████████ Elapsed: 4s | Loss: nan
1000/1000 [100%] ████████████████████

In [10]:
?inference.run

In [11]:
# %run weibull.py
approx_dict = {
    k: qk,
    p: qp
}

data_dict = {
    x2: y
}

In [12]:
losses_2 = []

for _ in range(100):
    inference = ed.KLqp(approx_dict, data = data_dict)
    inference.run()
    losses_2.append(inference.loss.eval())

1000/1000 [100%] ██████████████████████████████ Elapsed: 8s | Loss: 170.748
1000/1000 [100%] ██████████████████████████████ Elapsed: 8s | Loss: 433.151
1000/1000 [100%] ██████████████████████████████ Elapsed: 8s | Loss: 511.203
1000/1000 [100%] ██████████████████████████████ Elapsed: 8s | Loss: 518.099
1000/1000 [100%] ██████████████████████████████ Elapsed: 8s | Loss: 877.226
1000/1000 [100%] ██████████████████████████████ Elapsed: 8s | Loss: 146.960
1000/1000 [100%] ██████████████████████████████ Elapsed: 8s | Loss: 453.547
1000/1000 [100%] ██████████████████████████████ Elapsed: 8s | Loss: 138.349
1000/1000 [100%] ██████████████████████████████ Elapsed: 11s | Loss: 399.327
1000/1000 [100%] ██████████████████████████████ Elapsed: 8s | Loss: 401.938
1000/1000 [100%] ██████████████████████████████ Elapsed: 8s | Loss: 432.552
1000/1000 [100%] ██████████████████████████████ Elapsed: 8s | Loss: 280.705
1000/1000 [100%] ██████████████████████████████ Elapsed: 8s | Loss: 209.541
1000/1000 [

In [14]:
print(np.count_nonzero(np.isnan(np.array(losses_1))))
print(np.count_nonzero(np.isnan(np.array(losses_2))))

46
7
